In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import ta
import requests

from matplotlib import rcParams
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.widgets import RadioButtons
from matplotlib.animation import FuncAnimation
from matplotlib.dates import DateFormatter
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np

In [ ]:
tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'AVGO', 'ADBE', 'CRM',
    'CSCO', 'INTC', 'AMD', 'QCOM', 'ORCL', 'TXN', 'INTU', 'AMAT', 'MU', 'NOW',
    'SHOP', 'PANW', 'SNOW', 'ZM', 'PLTR', 'UBER', 'LYFT', 'DOCU', 'FSLY', 'TWLO'
]


In [31]:
API_KEY = "B6WMMM9S4ZYE4KRT"

In [32]:
import pandas as pd
import requests
import time
from tqdm import tqdm
import ta

# Assuming API_KEY is defined
API_KEY = 'your_api_key'

def fetch_alpha_vantage_stock(ticker, interval='1min', outputsize='full'):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_INTRADAY',
        'symbol': ticker,
        'interval': interval,
        'outputsize': outputsize,
        'apikey': API_KEY
    }

    r = requests.get(url, params=params)
    data = r.json()

    # Check for error messages or empty data
    if "Error Message" in data or "Note" in data:
        print(f"Error for {ticker}: {data.get('Error Message', data.get('Note'))}")
        return None
    
    key = f'Time Series ({interval})'
    if key not in data:
        print(f"No data for {ticker}")
        print(f"Full response: {data}")
        return None

    df = pd.DataFrame(data[key]).T
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df.astype(float)
    df['Ticker'] = ticker

    # Compute additional columns
    df['Returns'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['EMA_2'] = df['Close'].ewm(span=2, adjust=False).mean()

    rsi = ta.momentum.RSIIndicator(close=df['Close'])
    df['RSI'] = rsi.rsi()

    macd = ta.trend.MACD(close=df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Diff'] = macd.macd_diff()

    bb = ta.volatility.BollingerBands(close=df['Close'])
    df['BB_High'] = bb.bollinger_hband()
    df['BB_Low'] = bb.bollinger_lband()

    df['Volume_EMA'] = df['Volume'].ewm(span=20).mean()
    df['Volatility'] = df['Log_Returns'].rolling(window=60).std()

    df.dropna(inplace=True)
    return df


def fetch_all_alpha_stocks(tickers, output_csv=None):
    all_data = []

    for ticker in tqdm(tickers):
        df = fetch_alpha_vantage_stock(ticker)
        if df is not None:
            all_data.append(df)
        
        # Avoid hitting the API rate limit
        time.sleep(12)  # Sleep for 12 seconds between requests to avoid rate limits

    combined_df = pd.concat(all_data)
    combined_df.index.names = ['Datetime']
    combined_df = combined_df.sort_index()
    
    # If an output CSV file name is provided, save the data to a CSV
    if output_csv:
        combined_df.to_csv(output_csv)
        print(f"Data saved to {output_csv}")

    return combined_df


# Example usage:
tickers = ['AAPL', 'MSFT']
df = fetch_all_alpha_stocks(tickers, output_csv='stock_data.csv')

# This will save the combined dataframe to 'stock_data.csv' locally in the working directory


  0%|          | 0/2 [00:00<?, ?it/s]

No data for AAPL
Full response: {'Information': 'We have detected your API key as your_api_key and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


 50%|█████     | 1/2 [00:12<00:12, 12.74s/it]

No data for MSFT
Full response: {'Information': 'We have detected your API key as your_api_key and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


100%|██████████| 2/2 [00:25<00:00, 12.75s/it]


ValueError: No objects to concatenate

Plotting the data to aurafarm

In [ ]:
# Dark Mode Setup
rcParams['figure.facecolor'] = '#1a1a1a'
rcParams['axes.facecolor'] = '#2a2a2a'
rcParams['axes.edgecolor'] = 'white'
rcParams['text.color'] = 'white'
rcParams['xtick.color'] = 'white'
rcParams['ytick.color'] = 'white'

# -- CORRELATION HEATMAP --
def plot_correlation_heatmap(df):
    pivot = df.pivot_table(index='Datetime', columns='Ticker', values='Close')
    corr = pivot.corr()

    #fig, ax = plt.subplots(figsize=(12, 8))
    sns.heatmap(corr, annot=True, cmap='GnBu', vmin=-1, vmax=1, center=0)
    # ax.set_title('Ticker Correlation Heatmap', fontsize=16, pad=20)
    # plt.tight_layout()
    # plt.show()
    return corr

# -- ANIMATED BOLLINGER BANDS --
def animate_bollinger(ticker, df):
    ticker_df = df[df['Ticker'] == ticker].copy()
    ticker_df = ticker_df.sort_index()
    ticker_df = ticker_df.last('30D')  # Requires datetime index

    if ticker_df.empty:
        print(f"No data for {ticker} in the last 30 days.")
        return

    fig, ax = plt.subplots(figsize=(14, 7))
    ax.xaxis.set_major_formatter(DateFormatter('%m-%d %H:%M'))

    def update(frame):
        ax.clear()
        current_data = ticker_df.iloc[:frame]

        ax.plot(current_data.index, current_data['Close'], label='Price', color='royalblue')
        ax.plot(current_data.index, current_data['SMA_5'], label='SMA(5)', color='orange')
        ax.plot(current_data.index, current_data['BB_High'], label='Upper Band', linestyle='--', color='red')
        ax.plot(current_data.index, current_data['BB_Low'], label='Lower Band', linestyle='--', color='green')

        ax.fill_between(current_data.index,
                        current_data['BB_Low'],
                        current_data['BB_High'],
                        color='gray', alpha=0.2)

        ax.set_title(f'{ticker} Bollinger Bands Evolution', fontsize=16)
        ax.legend()
        plt.xticks(rotation=45)

    ani = FuncAnimation(fig, update, frames=len(ticker_df), interval=100)
    plt.tight_layout()
    plt.show()

# -- 3D VOLATILITY PLOT --
def plot_3d_volatility(df):
    fig = plt.figure(figsize=(16, 10))
    ax = fig.add_subplot(111, projection='3d')

    for ticker in df['Ticker'].unique():
        ticker_df = df[df['Ticker'] == ticker].copy()
        ticker_df = ticker_df.sort_index()
        if len(ticker_df) < 2:
            continue  # skip if not enough data

        dates = pd.to_datetime(ticker_df.index).astype(np.int64) // 10**9  # numeric for 3D plot
        ax.plot(dates,
                ticker_df['Volatility'],
                ticker_df['RSI'],
                label=ticker,
                linewidth=2)

    ax.set_xlabel('DateTime (numeric)', fontsize=12)
    ax.set_ylabel('Volatility', fontsize=12)
    ax.set_zlabel('RSI', fontsize=12)
    ax.set_title('3D Volatility-RSI Timeline', fontsize=16)
    ax.legend()
    ax.view_init(elev=25, azim=45)
    plt.tight_layout()
    plt.show()


In [112]:
df = fetch_all_alpha_stocks(tickers)

plot_correlation_heatmap(df)

 25%|██▌       | 1/4 [00:00<00:02,  1.34it/s]

No data for AAPL


 50%|█████     | 2/4 [00:01<00:00,  2.08it/s]

No data for MSFT


 75%|███████▌  | 3/4 [00:01<00:00,  1.69it/s]

No data for AMZN


100%|██████████| 4/4 [00:02<00:00,  1.59it/s]

No data for GOOG


ValueError: No objects to concatenate

In [ ]:
df = fetch_all_alpha_stocks(tickers)

for ticker in df:
    animate_bollinger(ticker, df)

In [ ]:
plot_3d_volatility(df)

In [ ]:
features = ['Close', 'Returns', 'Log_Returns', 'SMA_5', 'EMA_2', 'RSI', 'MACD', 'MACD_Signal', 'MACD_Diff', 'BB_High', 'BB_Low', 'Volume_EMA', 'Volatility']
df = fetch_all_alpha_stocks(tickers)

def scale_features(df, feature_cols):
    scalers = {}
    
    for ticker in df['Ticker'].unique():
        
        scaler = MinMaxScaler()
        
        idx = df['Ticker'] == ticker
        
        df.loc[idx, feature_cols] = scaler.fit_transform(df.loc[idx, feature_cols])
        
        scalers[ticker] = scaler
        
    return df, scalers



def create_sequences(df, feature_cols, sequence_length=60):
    X = []
    y = []


    for ticker in df['Ticker'].unique():
        
        df_ticker = df[df['Ticker'] == ticker]
        
        data = df_ticker[feature_cols].values
        target = df_ticker['Close'].values  # or next-day close/return etc.
        
        
        for i in range(sequence_length, len(df_ticker)):
            X.append(data[i-sequence_length:i])  # past 60 days
            y.append(target[i])  # predict next close price

    return np.array(X), np.array(y)

# Normalising the data
combined_df, scalers = scale_features(df, features)

# Creating sequences for the LSTM and such
X, y = create_sequences(combined_df, features, sequence_length=60) # 60 candlesticks = 1 Hour
